In [6]:
import pandas as pd
import json
import os
from datetime import datetime, timedelta

def process_json_to_csv(date_str):
    """
    Process a JSON boxscore file and convert to CSV format
    Captures ALL available batting statistics from the MLB Stats API
    """
    json_path = f'data/json/boxscores/mlb_boxscores_{date_str}.json'
    csv_output_path = f'data/csv/boxscores/mlb_boxscores_player_batting{date_str}.csv'
    
    if not os.path.exists(json_path):
        print(f"JSON file not found: {json_path}")
        return False
    
    # Load JSON data
    with open(json_path, 'r') as f:
        all_boxscores = json.load(f)
    
    # Define all available batting statistics fields (32 total)
    batting_stat_fields = [
        'gamesPlayed', 'flyOuts', 'groundOuts', 'airOuts', 'runs', 'doubles', 'triples', 
        'homeRuns', 'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch', 
        'atBats', 'caughtStealing', 'stolenBases', 'stolenBasePercentage', 'groundIntoDoublePlay', 
        'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi', 'leftOnBase', 
        'sacBunts', 'sacFlies', 'catchersInterference', 'pickoffs', 'atBatsPerHomeRun', 
        'popOuts', 'lineOuts', 'summary', 'note'
    ]
    
    # Convert to CSV - flatten batting data for all players
    batting_rows = []
    for game in all_boxscores:
        if game['boxscore']:
            for team_type in ['home', 'away']:
                team = game['boxscore']['teams'][team_type]
                batters = team.get('batters', [])
                players = team.get('players', {})
                
                for batter_id in batters:
                    player_key = f"ID{batter_id}"
                    player = players.get(player_key, {})
                    if player:
                        person = player.get('person', {})
                        stats = player.get('stats', {}).get('batting', {})
                        
                        # Base player and game information
                        row = {
                            'gamePk': game['gamePk'],
                            'team': team['team']['name'],
                            'team_type': team_type,
                            'player_id': person.get('id'),
                            'player_name': person.get('fullName'),
                            'batting_order': player.get('battingOrder'),
                            'position': player.get('position', {}).get('name', ''),
                            'position_code': player.get('position', {}).get('code', ''),
                            'jersey_number': person.get('primaryNumber', ''),
                        }
                        
                        # Add ALL batting statistics
                        for field in batting_stat_fields:
                            row[field] = stats.get(field)
                        
                        batting_rows.append(row)
    
    # Ensure output directory exists
    os.makedirs('data/csv/boxscores', exist_ok=True)
    
    # Create and save CSV
    df = pd.DataFrame(batting_rows)
    df.to_csv(csv_output_path, index=False)
    print(f"  - Saved {len(df)} batting records with {len(df.columns)} columns to {csv_output_path}")
    
    return df

In [7]:
# Define date range for processing (should match the JSON fetch range)
start_date = datetime(2025, 3, 27)
end_date = datetime(2025, 9, 30)

# Generate list of dates
date_range = []
current_date = start_date
while current_date <= end_date:
    date_range.append(current_date.strftime('%Y-%m-%d'))
    current_date += timedelta(days=1)

print(f"Processing JSON to CSV for {len(date_range)} dates")
print(f"Date range: {date_range[0]} to {date_range[-1]}")

Processing JSON to CSV for 188 dates
Date range: 2025-03-27 to 2025-09-30


In [9]:
# Process each date - convert JSON to CSV
all_batting_data = []
successful_conversions = []
failed_conversions = []

for date_str in date_range:
    print(f"Processing {date_str}...")
    
    try:
        daily_df = process_json_to_csv(date_str)
        
        if daily_df is not False and not daily_df.empty:
            # Add date column for tracking
            daily_df['date'] = date_str
            all_batting_data.append(daily_df)
            successful_conversions.append(date_str)
        else:
            failed_conversions.append(date_str)
        
    except Exception as e:
        print(f"  - Error processing {date_str}: {e}")
        failed_conversions.append(date_str)

print(f"\nConversion Summary:")
print(f"Successfully converted: {len(successful_conversions)} dates")
print(f"Failed: {len(failed_conversions)} dates")

if failed_conversions:
    print(f"Failed dates: {failed_conversions}")

Processing 2025-03-27...
  - Saved 420 batting records with 41 columns to data/csv/boxscores/mlb_boxscores_player_batting2025-03-27.csv
Processing 2025-03-28...
  - Saved 259 batting records with 41 columns to data/csv/boxscores/mlb_boxscores_player_batting2025-03-28.csv
Processing 2025-03-29...
  - Saved 467 batting records with 41 columns to data/csv/boxscores/mlb_boxscores_player_batting2025-03-29.csv
Processing 2025-03-30...
  - Saved 386 batting records with 41 columns to data/csv/boxscores/mlb_boxscores_player_batting2025-03-30.csv
Processing 2025-03-31...
  - Saved 415 batting records with 41 columns to data/csv/boxscores/mlb_boxscores_player_batting2025-03-31.csv
Processing 2025-04-01...
  - Saved 380 batting records with 41 columns to data/csv/boxscores/mlb_boxscores_player_batting2025-04-01.csv
Processing 2025-04-02...
  - Saved 386 batting records with 41 columns to data/csv/boxscores/mlb_boxscores_player_batting2025-03-30.csv
Processing 2025-03-31...
  - Saved 415 batting r

In [10]:
# Combine all batting data into one master CSV
if all_batting_data:
    combined_df = pd.concat(all_batting_data, ignore_index=True)
    
    # Create filename based on actual date range
    start_str = date_range[0]
    end_str = date_range[-1]
    combined_filename = f'data/csv/boxscores/mlb_player_batting_boxscores_{start_str}_to_{end_str}.csv'
    
    combined_df.to_csv(combined_filename, index=False)
    
    print(f"\nCombined Dataset Summary:")
    print(f"Total batting records: {len(combined_df):,}")
    print(f"Date range: {start_str} to {end_str}")
    print(f"Unique players: {combined_df['player_id'].nunique():,}")
    print(f"Unique games: {combined_df['gamePk'].nunique():,}")
    print(f"Combined file: {combined_filename}")
    
    # Display sample data
    print(f"\nSample data:")
    print(combined_df.head())
    
else:
    print("No batting data found for the date range")


Combined Dataset Summary:
Total batting records: 71,681
Date range: 2025-03-27 to 2025-09-30
Unique players: 1,472
Unique games: 2,428
Combined file: data/csv/boxscores/mlb_player_batting_boxscores_2025-03-27_to_2025-09-30.csv

Sample data:
   gamePk              team team_type  player_id        player_name  \
0  778557  New York Yankees      home     669224       Austin Wells   
1  778557  New York Yankees      home     592450        Aaron Judge   
2  778557  New York Yankees      home     641355     Cody Bellinger   
3  778557  New York Yankees      home     502671   Paul Goldschmidt   
4  778557  New York Yankees      home     665862  Jazz Chisholm Jr.   

  batting_order     position position_code jersey_number  gamesPlayed  ...  \
0           100      Catcher             2                        1.0  ...   
1           200   Outfielder             9                        1.0  ...   
2           300   Outfielder             7                        1.0  ...   
3           400   F

In [ ]:
# Display the complete column structure of the enhanced dataset
if all_batting_data:
    print("\nEnhanced Dataset Column Structure:")
    print("=" * 60)
    
    # Show all columns with their types
    column_info = []
    for col in combined_df.columns:
        col_type = str(combined_df[col].dtype)
        non_null_count = combined_df[col].count()
        null_count = len(combined_df) - non_null_count
        column_info.append({
            'Column': col,
            'Type': col_type,
            'Non-Null': non_null_count,
            'Null': null_count,
            'Sample': str(combined_df[col].iloc[0]) if non_null_count > 0 else 'N/A'
        })
    
    col_df = pd.DataFrame(column_info)
    print(f"\nTotal columns captured: {len(col_df)}")
    print(col_df.to_string(index=False, max_rows=50))
    
    # Show batting statistics specifically
    batting_cols = [col for col in combined_df.columns if col not in 
                   ['gamePk', 'team', 'team_type', 'player_id', 'player_name', 
                    'batting_order', 'position', 'position_code', 'jersey_number', 'date']]
    
    print(f"\nBatting Statistics Columns ({len(batting_cols)}):")
    for i, col in enumerate(batting_cols, 1):
        print(f"{i:2d}. {col}")

In [ ]:
# Optional: Display basic statistics about the combined dataset
if all_batting_data:
    print("\nDataset Statistics:")
    print("=" * 50)
    
    # Batting statistics summary
    numeric_cols = ['atBats', 'hits', 'runs', 'homeRuns', 'rbi', 'baseOnBalls', 'strikeOuts']
    stats_summary = combined_df[numeric_cols].describe()
    print("\nBatting Statistics Summary:")
    print(stats_summary)
    
    # Team breakdown
    print(f"\nTeam Breakdown:")
    team_summary = combined_df['team'].value_counts().head(10)
    print(team_summary)
    
    # Date breakdown
    print(f"\nDaily Record Counts:")
    date_summary = combined_df['date'].value_counts().sort_index()
    print(date_summary)